In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import jax 
import jax.config 
jax.config.update('jax_enable_x64', True)
import jax.numpy as jnp

import numpy as np

import dp_matrix_factorization.loops as loops

In [14]:
dimensionality = 10
s_matrix = np.tril(np.ones(shape=(dimensionality, dimensionality)))

In [15]:
results = loops.compute_h_fixed_point_iteration(
    s_matrix=jnp.array(s_matrix),
    rtol=1e-3,
    max_iterations=100,
)

In [21]:
C = results["H"]
B = results["W"]

In [22]:
B @ C

Array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float64)

In [26]:
max_col_norm_C = np.max(np.sqrt((C**2).sum(axis=1)))
max_data_abs_value = 1
sensitivity = 2 * max_col_norm_C * max_data_abs_value